# **DS501 Class Project**
**Authors:** Daniel Alvarado Segura and Jun Lee

**Description**: In this project we plan to merge two data sets which include housing selling prices for California. With this new merged data set we will be able to find a much more refined image of the house market in the area of California. 

**Data Sets**: We will use two open source data sets which inlcude information about the housing market of the area of California. The first set contains information of the houses per quadrant based on longitude and latitude. The second set has information of specific houses and their specific address. 

# **Data Loading and Cleaning**
**Process:** To be able to merge this two data bases we must first clean the data by dropping any rows with key information such as its address/location and house features such as number of rooms and proximity to the coast. Additionally, the second data set is split between a training set and a testing set. We will concatenate the contents of these two, to then be able to merge it with the first data set. Additionally, for the purpose of finding the information we want we will drop any rows with data that do not belong to the counties of Los Angeles, San Diego, Orange, Riverside, and San Bernardino.

**Framework:** Our data processing framework is composed of 4 main components, and depending on the dataset we may add others. Here is a small explanation of the main components:

1. Required data definition (main funtion): The way our data processing/cleaning works is we specify which counties we want informatio from. Furthermore, in case you have a set that was divided in subsets, we can put them back together (assuming they have the same features) and specify which features to be used to compare with the other database. 

2. Set completion (complete_set# function): In this function we do the preliminary data cleaning necesary to execute the next 2 main components. This is tailored to our datasets, but it can easily be changed as it is it's own function.

3. Find county (get_county funtion): With this function we utilize Geocoding to find the county each row of data belongs to. This allows us to filter out data from counties we don't want to analyze. In the case of the second dataset there is an additional step before this one, which is get_coordinates that finds the longitude and latitude of each address to then use this fucntion.

4. County Filter (county_filter funtion): This function drops rows from counties we don't want to analyze. Said counties are decided beforehand and are provided to the function via a dictionary. 

**Functions:**

In [1]:
#Necesary modules
import pandas as pd
import requests
from matplotlib import pyplot as plt

C:\Users\danie\AppData\Local\Temp\ipykernel_38292\3869539251.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Function to get county from geocode
def get_county(latitude, longitude):
    try:
        geocode_api = f"https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x={longitude}&y={latitude}&benchmark=Public_AR_Census2020&vintage=Census2020_Census2020&layers=6&format=json"
        response = requests.get(geocode_api)
        response.raise_for_status()  # Raise an exception for 4xx or 5xx status codes
        geocode_api_data = response.json()
        county_code = geocode_api_data['result']['geographies']['Census Tracts'][0]['COUNTY']
        return county_code
    except (requests.RequestException, KeyError, IndexError) as e:
        print(f"Error occurred for coordinates ({longitude}, {latitude}): {str(e)}")
        return None

In [3]:
#Function to filter data from wanted counties
def county_filter(filtered_data, county_names):
    # Iterate over each county code in the 'county' column
    indices_to_drop = []  # List to store indices of rows to drop
    for index, county_code in filtered_data['county'].items():
        # Check if the county code exists in the dictionary keys
        if county_code in county_names:
            # Replace the county code with the corresponding county name
            filtered_data.at[index, 'county'] = county_names[county_code]
        else:
            # Add the index to the list of indices to drop
            indices_to_drop.append(index)

        # Drop rows with indices from the list and round values
        filtered_data.drop(indices_to_drop, inplace=True)
    return filtered_data

In [4]:
# Function to get geocode (longitude and latitude) from address
def get_coordinates(address, city, zip_code, state):
    # Format the address for the API request
    formatted_address = f"{address.replace(' ', '+')},+{city.replace(' ', '+')},+{state}+{zip_code}"
    formatted_address = formatted_address.replace('#','')
    geocoding_url = f"https://geocoding.geo.census.gov/geocoder/locations/onelineaddress?address={formatted_address}&benchmark=2020&format=json"
    
    try:
        response = requests.get(geocoding_url)
        response.raise_for_status()  # Raise an exception for 4xx or 5xx status codes
        geocoding_data = response.json()
        
        # Extract latitude and longitude from the API response
        if 'result' in geocoding_data and 'addressMatches' in geocoding_data['result'] and len(geocoding_data['result']['addressMatches']) > 0:
            latitude = geocoding_data['result']['addressMatches'][0]['coordinates']['y']
            longitude = geocoding_data['result']['addressMatches'][0]['coordinates']['x']
            return latitude, longitude
        else:
            return None, None
    except requests.RequestException as e:
        print(f"Error occurred while getting geocode for address {formatted_address}: {e}")
        return None, None

In [5]:
#Function to aggregate desired columns in dataset 2
def data_aggregation(full_data,aggregation_functions):
    mod_full_data = full_data.groupby(['longitude', 'latitude']).agg(aggregation_functions).reset_index()

**Data Loading of First Data Set**

In [ ]:
#Function to process a dataSet
def complete_set1(file_path):
    coordinate_based_data = pd.read_csv(file_path)
    cleaned_coordinate_based_data = coordinate_based_data.dropna(subset=['longitude', 'latitude'])
    filtered_data = cleaned_coordinate_based_data[(cleaned_coordinate_based_data['longitude'] > -118.96) &
                                                  (cleaned_coordinate_based_data['latitude'] < 35.82)]

    duplicate_coords = filtered_data.groupby(['longitude', 'latitude']).size() > 1
    for coords, is_duplicate in duplicate_coords.items():
        if is_duplicate:
            avg_values = filtered_data.loc[(filtered_data['longitude'] == coords[0]) &
                                           (filtered_data['latitude'] == coords[1])].select_dtypes(include='number').mean()
            for col in avg_values.index:
                filtered_data.loc[(filtered_data['longitude'] == coords[0]) &
                                  (filtered_data['latitude'] == coords[1]), col] = avg_values[col]
    filtered_data.drop_duplicates(subset=['longitude', 'latitude'], keep='first', inplace=True)

    return filtered_data

#File path to dataSet
file_path = "local_path/California_House_Info.csv"

# Define County codes wanted
county_names = {
    '037': 'Los Angeles',
    '073': 'San Diego',
    '059': 'Orange',
    '065': 'Riverside',
    '071': 'San Bernardino'
}
#Load and Clean data for merging
cleaned_dataset1 = complete_set1(file_path)
cleaned_dataset1['county'] = cleaned_dataset1.apply(lambda row: get_county(row['latitude'], row['longitude']), axis=1)
cleaned_dataset1 = county_filter(cleaned_dataset1, county_names)

cleaned_dataset1.to_csv('cleaned_dataset1.csv', index=False)
print(cleaned_dataset1)



**Data Loading of Second Data Set**

In [ ]:
def complete_set2(susbset_list):
    full_data = pd.DataFrame()
    for idx, file_name in enumerate(susbset_list):
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_name)
        if idx == 0:
            full_data = df
        else:
            full_data = pd.concat([full_data, df], ignore_index=True)

    # Clean the DataFrame
    full_data = full_data.dropna(subset=['Address', 'Bedrooms'])

    #Get coordinates (Longitude and Latitude)
    full_data['latitude'], full_data['longitude'] = zip(*full_data.apply(lambda row: get_coordinates(row['Address'], row['City'], row['Zip'], row['State']), axis=1))
    full_data['latitude'] = full_data['latitude'].round(2)
    full_data['longitude'] = full_data['longitude'].round(2)

    #Get counties
    full_data['county'] = full_data.apply(lambda row: get_county(row['latitude'], row['longitude']), axis=1)

    return full_data

def clean_set2(full_data, county_names, aggregation_functions):
    filtered_data = county_filter(full_data,county_names)
    cleaned_dataset2 = data_aggregation(full_data,aggregation_functions)

    return cleaned_dataset2



# List of subsets to join for dataSet
subset_list = ['train.csv', 'test.csv']

# Define County codes wanted
county_names = {
    '037': 'Los Angeles',
    '073': 'San Diego',
    '059': 'Orange',
    '065': 'Riverside',
    '071': 'San Bernardino'
}

# Agregate data for merging
# Define aggregation dictionary for desired columns
aggregation_functions = {
    'Sold Price': 'mean',  # Average price of homes - Use mean to get average of values in same area
    'Year built': 'mean',
    'Bedrooms': 'sum',  # Total number of bedrooms - Use sum to add values in same area
    'Bathrooms': 'sum',
    'Full bathrooms': 'sum',
    'Total interior livable area': 'mean', #Average house area
    'Elementary School Distance': 'mean', # Average distance to schools
    'Middle School Distance': 'mean', # Average distance to schools
    'High School Distance': 'mean', # Average distance to schools
    'Last Sold Price': 'mean'  #Average house price
}

# Get complete set 2 (join subsets, find coordinates and counties)
full_data = complete_set2(subset_list)

# Clean data
cleaned_dataset2 = clean_set2(full_data, county_names)

#Save csv of completed processed data set to current directory
cleaned_dataset2.to_csv('cleaned_dataset2.csv', index=False)
print(cleaned_dataset2)